In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import re
import h5py
import math

import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly_resampler

In [ ]:
with h5py.File('/data/IDLab/aar_foundation_models/processed_data/horse_1_ds_0.hdf5', 'r') as f:
    sr = f.attrs['sr']
    X_acc = f['acc'][:]
    X_gyr = f['gyr'][:]


In [26]:
n_samples = len(X_acc)
timesteps = np.linspace(0, math.ceil(n_samples / sr), n_samples, endpoint=False)

g = 9.81
X_acc_g = X_acc / g

fig = make_subplots(rows=3, cols=1, shared_xaxes=True)

colors = ['red', 'green', 'blue']
axis_labels = ['X', 'Y', 'Z']

for i in range(3):
    fig.add_trace(
        go.Scattergl(
            x=timesteps,
            y=X_acc_g[:, i],
            name=f'Acc {axis_labels[i]}',
            line=dict(color=colors[i])
        ),
        row=i+1, col=1
    )
    fig.update_yaxes(title_text="Acceleration (G)", row=i+1, col=1)

fig.update_layout(
    title_text="Accelerometer Traces",
    height=700
)
fig.update_xaxes(title_text="Time (s)", row=3, col=1)

fig = plotly_resampler.FigureWidgetResampler(fig)

fig

FigureWidgetResampler({
    'data': [{'line': {'color': 'red'},
              'name': '<b style="color:sandybrown">[R]</b> Acc X <i style="color:#fc9944">~10</i>',
              'type': 'scattergl',
              'uid': '787f599c-cbc4-4c59-9cd6-6b2cc3eb2a9a',
              'x': array([0.00000000e+00, 1.81004315e+00, 1.77304227e+01, ..., 1.04625994e+04,
                          1.04767698e+04, 1.04859900e+04], shape=(1000,)),
              'xaxis': 'x',
              'y': array([-0.60430733, -1.79437297, -0.18988332, ..., -0.88254253, -0.59503282,
                          -0.98993155], shape=(1000,)),
              'yaxis': 'y'},
             {'line': {'color': 'green'},
              'name': '<b style="color:sandybrown">[R]</b> Acc Y <i style="color:#fc9944">~10</i>',
              'type': 'scattergl',
              'uid': 'cb017874-fd34-4e2d-90e2-3ec7b9b2d377',
              'x': array([0.00000000e+00, 1.81004315e+00, 1.22302916e+01, ..., 1.04610994e+04,
                          1.

In [12]:
X_acc[:, 0]

array([-5.92825488, -5.47334033, -4.97532861, ..., -9.99375439,
       -9.88361719, -9.71122852], shape=(1048575,))